In [2]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding

# ---------------------------
# Sample data
# ---------------------------
input_texts = ['I love NLP', 'He plays football']
target_texts = [['PRON', 'VERB', 'NOUN'], ['PRON', 'VERB', 'NOUN']]

# ---------------------------
# Tokenization
# ---------------------------
word_vocab = sorted(set(word for sent in input_texts for word in sent.split()))
tag_vocab = sorted(set(tag for tags in target_texts for tag in tags))

word2idx = {word: i+1 for i, word in enumerate(word_vocab)}   # start indexing at 1
tag2idx = {tag: i+1 for i, tag in enumerate(tag_vocab)}       # start indexing at 1
idx2tag = {i: tag for tag, i in tag2idx.items()}

# Encoder input (word indices)
encoder_input_data = np.array([
    [word2idx[word] for word in sent.split()] for sent in input_texts
])

# Decoder input (tag indices shifted by <START>)
decoder_input_data = np.array([
    [tag2idx[tag] for tag in tags] for tags in target_texts
])

# Decoder output (same as input but shifted for prediction)
decoder_output_data = np.expand_dims(decoder_input_data, -1)  # shape (batch, seq_len, 1)

# ---------------------------
# Model
# ---------------------------
# Encoder
encoder_inputs = Input(shape=(None,))
x = Embedding(input_dim=len(word_vocab) + 1, output_dim=64)(encoder_inputs)
encoder = LSTM(64, return_state=True)
encoder_outputs, state_h, state_c = encoder(x)

# Decoder
decoder_inputs = Input(shape=(None,))
y = Embedding(input_dim=len(tag_vocab) + 1, output_dim=64)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(y, initial_state=[state_h, state_c])

decoder_dense = Dense(len(tag_vocab) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model definition
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# ---------------------------
# Training
# ---------------------------
model.fit([encoder_input_data, decoder_input_data],
          decoder_output_data,
          epochs=50,
          verbose=1)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6667 - loss: 1.3805
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8333 - loss: 1.3726
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8333 - loss: 1.3646
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 1.0000 - loss: 1.3564
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 1.0000 - loss: 1.3479
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 1.0000 - loss: 1.3391
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 1.0000 - loss: 1.3299
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 1.0000 - loss: 1.3201
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 1.0000 - loss: 1.3098
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 1.0000 - loss: 1.2987
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 1.0000 - loss: 1.2870
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 1.2744
Epo